# Guided Backpropagation Saliency Map 

#### Link to Readme section:    

https://git.cs.vt.edu/sdeepti/facial-expression-recognition/-/blob/main/README.md#saliency-maps

#### Citations:

- https://github.com/utkuozbulak/pytorch-cnn-visualizations#gradient-visualization

**Motivation:** One of the best ways to interpret and visualize the CNN model is through saliency maps. Saliency maps are a way to measure the spatial support of a particular class in each image. It is a visualization technique to gain better insight into the decision-making of the CNN and helps to highlight what each layer of a convolutional layer focuses on.

Since the model's interpretability was not as clear using Vanilla Backpropagation (it was a very noisy image), we decided to use another approach which was Guided-Backpropagation Saliency. Guided Backpropagation Saliency combines the previously used Vanilla Backpropagation technique at ReLUs with DeconvNets. Guided backpropagation visualizes gradients with respect to the image where negative gradients are suppressed when backpropagating through ReLU layers.

#### 1. Initial Set-Up

This adds all the imports that are necessary for the code to run smoothly. It involves importing 'torch' which is necessary to work with our model and retrieve our datasets. Additionally, 'matplotlib.cm' is imported to utilize the 'mpl_color_map' feature so that we can use colormaps. 

In [ ]:
import os
import copy
import numpy as np
from PIL import Image
import matplotlib.cm as mpl_color_map
from matplotlib.colors import ListedColormap
from matplotlib import pyplot as plt

import torch
from torch.autograd import Variable
from torchvision import models
import torch
from torch.nn import ReLU

#### 2. Write the GuidedBackprop class.

The code below is a class called GuidedBackProp which produces gradients generated with guided back propagation from the given image. 

In [ ]:
class GuidedBackprop():
	"""
	   Produces gradients generated with guided back propagation from the given image
	"""
	def __init__(self, model):
		self.model = model
		self.gradients = None
		self.forward_relu_outputs = []

		# Put model in evaluation mode
		self.model.eval()
		self.update_relus()
		self.hook_layers()

	def hook_layers(self):
		def hook_function(module, grad_in, grad_out):
			self.gradients = grad_in[0]
		# Register hook to the first layer
		first_layer = list(self.model.children())[0]
		first_layer.register_backward_hook(hook_function)

	def update_relus(self):
		"""
			Updates relu activation functions so that it only returns positive gradients
		"""
		def relu_hook_function(module, grad_in, grad_out):
			"""
			If there is a negative gradient, changes it to zero
			"""
			if isinstance(module, ReLU):
				return (torch.clamp(grad_in[0], min=0.0),)

		# Loop through layers, hook up ReLUs with relu_hook_function
		for module in self.model.modules():
			if isinstance(module, ReLU):
				module.register_backward_hook(relu_hook_function)

	def generate_gradients(self, input_image, target_class):
		# Forward pass
		model_output = self.model(input_image)

		# Zero gradients
		self.model.zero_grad()

		# Target for backprop
		one_hot_output = torch.FloatTensor(1, model_output.size()[-1]).zero_()
		one_hot_output[0][target_class] = 1

		# Backward pass
		model_output.backward(gradient=one_hot_output)

		# Convert Pytorch variable to numpy array
		# [0] to get rid of the first channel (1,3,224,224)
		gradients_as_arr = self.gradients.data.numpy()[0]
		return gradients_as_arr

#### 3. Perform Guided Backpropagation on the model.

The code below calls the GuidedBackprop class that we created, and passes the model to it. Then we retrieve the model's output after passing image. Backpropagation is then done to get the derivative of the output based on the image. The colored gradients are saved before converting them to grayscale, and the resulting grayscale gradients are saved. The positive and negative saliency maps are then plotted.

In [ ]:
# Guided backprop
target_example = 0
(original_image, prep_img, target_class, file_name_to_export, pretrained_model) =\
  get_params(target_example, model)

GBP = GuidedBackprop(model)

# Get gradients
# Retrieve output from the image
output = model(image)

# # Do backpropagation to get the derivative of the output based on the image
guided_grads = GBP.generate_gradients(prep_img, target_class)

# Save colored gradients
save_gradient_images(guided_grads, file_name_to_export + '_Guided_BP_color')

# Convert to grayscale
grayscale_guided_grads = convert_to_grayscale(guided_grads)

# Save grayscale gradients
save_gradient_images(grayscale_guided_grads, file_name_to_export + '_Guided_BP_gray')

# Positive and negative saliency maps
pos_sal, neg_sal = get_positive_negative_saliency(guided_grads)
save_gradient_images(pos_sal, file_name_to_export + '_pos_sal')
save_gradient_images(neg_sal, file_name_to_export + '_neg_sal')

print('Guided backprop completed')

|Guided Backpropagation Saliency|Colored Guided Backpropagation|Guided Backpropagation Negative Saliency|Guided Backpropagation Positive Saliency|
| ------ | ------ | ------ | ------ |
| <img src="https://git.cs.vt.edu/sdeepti/facial-expression-recognition/-/raw/main/Images/AF01HAHR_Guided_BP_gray.png"> | <img src="https://git.cs.vt.edu/sdeepti/facial-expression-recognition/-/raw/main/Images/AF01HAHR_Guided_BP_color.png"> | <img src="https://git.cs.vt.edu/sdeepti/facial-expression-recognition/-/raw/main/Images/AF01HAHR_neg_sal.png"> | <img src="https://git.cs.vt.edu/sdeepti/facial-expression-recognition/-/raw/main/Images/AF01HAHR_pos_sal.png"> |

